In [ ]:
# -*- Encoding:UTF-8 -*-
'''
@author: Jason.F
@data: 2019.07.22
@function: Implementing BMF(Bayesian Matrix Factorization) By VI
           Dataset: Pinterest-20
           Evaluating: hitradio,ndcg
'''
import sys
import time
import logging

import pymc3 as pm
import numpy as np
import pandas as pd
import theano
import theano.tensor as t
import heapq
import math

def getTraindata():
    data = []
    filePath = '/data/fjsdata/ctKngBase/ml/pinterest-20.train.rating'
    u = 0
    i = 0
    maxr = 0.0
    with open(filePath, 'r') as f:
        for line in f:
            if line:
                lines = line[:-1].split("\t")
                user = int(lines[0])
                item = int(lines[1])
                score = float(lines[2])
                data.append((user, item, score))
                if user > u: u = user
                if item > i: i = item
                if score > maxr: maxr = score
    print("Loading Success!\n"
                  "Data Info:\n"
                  "\tUser Num: {}\n"
                  "\tItem Num: {}\n"
                  "\tData Size: {}".format(u, i, len(data)))
    
    R = np.zeros([u+1, i+1], dtype=np.float32)
    for i in data:
        user = i[0]
        item = i[1]
        rating = i[2]
        R[user][item] = rating
    return R

def getTestdata():
    testset = []
    filePath = '/data/fjsdata/ctKngBase/ml/pinterest-20.test.negative'
    with open(filePath, 'r') as fd:
        line = fd.readline()
        while line != None and line != '':
            arr = line.split('\t')
            u = eval(arr[0])[0]
            testset.append([u, eval(arr[0])[1]])#one postive item
            for i in arr[1:]:
                testset.append([u, int(i)]) #99 negative items
            line = fd.readline()
    return testset

def getHitRatio(ranklist, targetItem):
    for item in ranklist:
        if item == targetItem:
            return 1
    return 0
def getNDCG(ranklist, targetItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == targetItem:
            return math.log(2) / math.log(i+2)
    return 0

def build_BMF(R, K, alpha=2, std=0.01):
    
    alpha_u = 1 / R.var(axis=1).mean()
    alpha_v = 1 / R.var(axis=0).mean()

    logging.info('building the BMF model')
    n, m = R.shape
    with pm.Model() as bmf:
        U = pm.MvNormal('U', mu=0, tau=alpha_u * np.eye(K), shape=(n, K), testval=np.random.randn(n, K) * std)
        V = pm.MvNormal('V', mu=0, tau=alpha_v * np.eye(K), shape=(m, K), testval=np.random.randn(m, K) * std)
        nR = pm.Normal('nR', mu=t.dot(U, V.T), tau=alpha * np.ones(R.shape),observed=R)
    logging.info('done building BMF model')
    return bmf

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO,format='[%(asctime)s]: %(message)s')
    
    # Read data and build BMF model.
    R = getTraindata()
    bmf = build_BMF(R, K=64)#dim is the number of latent factors

    with bmf:# sample with BMF
        tstart = time.time()
        logging.info('Start BMF sampling')
        inference = pm.ADVI()
        approx = pm.fit(n=1000, method=inference)
        trace = approx.sample(draws=500)
        elapsed = time.time() - tstart    
        logging.info('Complete BMF sampling in %d seconds' % int(elapsed))
        
    with bmf:#evaluation
        testset = getTestdata()
        ppc = pm.sample_posterior_predictive(trace, progressbar=True)
        nR = np.mean(ppc['nR'],0)#three dims, calcuate the mean with the first dim for posterior
        hits = []
        ndcgs = []
        prev_u = testset[0][0]
        pos_i = testset[0][1]
        scorelist = []
        for u, i in testset:
            if prev_u == u:
                scorelist.append([i,nR[u,i]])
            else:
                map_item_score = {}
                for item, rate in scorelist: #turn dict
                    map_item_score[item] = rate
                ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
                hr = getHitRatio(ranklist, pos_i)
                hits.append(hr)
                ndcg = getNDCG(ranklist, pos_i)
                ndcgs.append(ndcg)
                #next user
                scorelist = []
                prev_u = u
                pos_i = i
                scorelist.append([i,nR[u,i]])
        hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
        print("HR@10: {}, NDCG@10: {}, At K {}".format(hitratio, ndcg, 64))

Loading Success!
Data Info:
	User Num: 55186
	Item Num: 9915
	Data Size: 1445622


[2019-07-24 02:10:26,486]: building the BMF model
[2019-07-24 02:41:46,664]: done building BMF model
[2019-07-24 02:41:46,726]: Start BMF sampling
Average Loss = 8.0852e+09:  29%|██▊       | 286/1000 [6:50:06<14:23:39, 72.58s/it] 
Interrupted at 286 [28%]: Average Loss = 8.0742e+09
[2019-07-24 09:42:26,364]: Interrupted at 286 [28%]: Average Loss = 8.0742e+09


In [3]:
# -*- Encoding:UTF-8 -*-
'''
@author: Jason.F
@data: 2019.07.22
@function: Implementing BMF(Bayesian Matrix Factorization) By VI
           Dataset: Movielen Dataset(ml-1m) 
           Evaluating: hitradio,ndcg
'''
import sys
import time
import logging

import pymc3 as pm
import numpy as np
import pandas as pd
import theano
import theano.tensor as t
import heapq
import math

def getTraindata():
    data = []
    filePath = '/data/fjsdata/ctKngBase/ml/ml-1m.train.rating'
    u = 0
    i = 0
    maxr = 0.0
    with open(filePath, 'r') as f:
        for line in f:
            if line:
                lines = line[:-1].split("\t")
                user = int(lines[0])
                item = int(lines[1])
                score = float(lines[2])
                data.append((user, item, score))
                if user > u: u = user
                if item > i: i = item
                if score > maxr: maxr = score
    print("Loading Success!\n"
                  "Data Info:\n"
                  "\tUser Num: {}\n"
                  "\tItem Num: {}\n"
                  "\tData Size: {}".format(u, i, len(data)))
    
    R = np.zeros([u+1, i+1], dtype=np.float32)
    for i in data:
        user = i[0]
        item = i[1]
        rating = i[2]
        R[user][item] = rating
    return R

def getTestdata():
    testset = []
    filePath = '/data/fjsdata/ctKngBase/ml/ml-1m.test.negative'
    with open(filePath, 'r') as fd:
        line = fd.readline()
        while line != None and line != '':
            arr = line.split('\t')
            u = eval(arr[0])[0]
            testset.append([u, eval(arr[0])[1]])#one postive item
            for i in arr[1:]:
                testset.append([u, int(i)]) #99 negative items
            line = fd.readline()
    return testset

def getHitRatio(ranklist, targetItem):
    for item in ranklist:
        if item == targetItem:
            return 1
    return 0
def getNDCG(ranklist, targetItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == targetItem:
            return math.log(2) / math.log(i+2)
    return 0

def build_BMF(R, K, alpha=2, std=0.01):
    
    alpha_u = 1 / R.var(axis=1).mean()
    alpha_v = 1 / R.var(axis=0).mean()

    logging.info('building the BMF model')
    n, m = R.shape
    with pm.Model() as bmf:
        U = pm.MvNormal('U', mu=0, tau=alpha_u * np.eye(K), shape=(n, K), testval=np.random.randn(n, K) * std)
        V = pm.MvNormal('V', mu=0, tau=alpha_v * np.eye(K), shape=(m, K), testval=np.random.randn(m, K) * std)
        nR = pm.Normal('nR', mu=t.dot(U, V.T), tau=alpha * np.ones(R.shape),observed=R)
    logging.info('done building BMF model')
    return bmf

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO,format='[%(asctime)s]: %(message)s')
    
    # Read data and build BMF model.
    R = getTraindata()
    bmf = build_BMF(R, K=64)#dim is the number of latent factors

    with bmf:# sample with BMF
        tstart = time.time()
        logging.info('Start BMF sampling')
        inference = pm.ADVI()
        approx = pm.fit(n=1000, method=inference)
        trace = approx.sample(draws=500)
        elapsed = time.time() - tstart    
        logging.info('Complete BMF sampling in %d seconds' % int(elapsed))
        
    with bmf:#evaluation
        testset = getTestdata()
        ppc = pm.sample_posterior_predictive(trace, progressbar=True)
        nR = np.mean(ppc['nR'],0)#three dims, calcuate the mean with the first dim for posterior
        hits = []
        ndcgs = []
        prev_u = testset[0][0]
        pos_i = testset[0][1]
        scorelist = []
        for u, i in testset:
            if prev_u == u:
                scorelist.append([i,nR[u,i]])
            else:
                map_item_score = {}
                for item, rate in scorelist: #turn dict
                    map_item_score[item] = rate
                ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
                hr = getHitRatio(ranklist, pos_i)
                hits.append(hr)
                ndcg = getNDCG(ranklist, pos_i)
                ndcgs.append(ndcg)
                #next user
                scorelist = []
                prev_u = u
                pos_i = i
                scorelist.append([i,nR[u,i]])
        hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
        print("HR@10: {}, NDCG@10: {}, At K {}".format(hitratio, ndcg, 64))

Loading Success!
Data Info:
	User Num: 6039
	Item Num: 3705
	Data Size: 994169


[2019-07-23 23:27:45,431]: building the BMF model
[2019-07-23 23:28:43,841]: done building BMF model
[2019-07-23 23:28:43,846]: Start BMF sampling
Average Loss = 2.7484e+08: 100%|██████████| 1000/1000 [53:55<00:00,  3.06s/it]
Finished [100%]: Average Loss = 2.7423e+08
[2019-07-24 00:23:00,032]: Finished [100%]: Average Loss = 2.7423e+08
[2019-07-24 00:24:37,526]: Complete BMF sampling in 3353 seconds
100%|██████████| 500/500 [1:13:32<00:00,  7.07s/it]


HR@10: 0.10200364298724955, NDCG@10: 0.04524941138321496, At K 64


In [7]:
#矩阵分解R=PQ，推荐概率模型MCMC采样-直接矩阵采样
import theano
import pymc3 as pm
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error
import theano.tensor as tt
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder

#1.数据集处理
#http://files.grouplens.org/datasets/movielens/ml-20m-README.html
#the following format of file ratings.csv: userId,movieId,rating,timestamp
#The lines within this file are ordered first by userId, then, within user, by movieId.
#Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).
#Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.
data = pd.read_csv("/data/fjsdata/BayesianRS/ml-20m/ratings.csv",sep=',',low_memory=False,iterator =True)
data = data.get_chunk(1000)
#将userId和movieId全部标准编号
le = LabelEncoder()
data = data.apply(le.fit_transform)
#2.构建U-I矩阵
uNum = len(data['userId'].unique())#统计用户数
iNum = len(data['movieId'].unique())#统计电影数
UI = np.zeros((uNum, iNum))#转成R矩阵，非常稀疏
for index, row in data.iterrows(): # 获取每行的值
    UI[int(row['userId'])][int(row['movieId'])] = row['rating']
#2.构建概率模型
#概率模型参数设置
mean= data['rating'].max()/2 #正态分布的均值和方差
k = 100 #隐因子数
Y_output = theano.shared(UI)#转numpy array
with pm.Model() as BMF_model:
    # Creating the model
    P = pm.Normal('P', mu=mean, sd=mean, shape=(uNum,k))
    Q = pm.Normal('Q', mu=mean, sd=mean, shape=(k,iNum))
    R = pm.Deterministic('R', tt.dot(P,Q))
    Y = pm.Normal('Y',mu=R, sd=mean, observed=Y_output)
#3.后验分布计算  
with BMF_model:        
    start=pm.find_MAP()  # 参数初猜
    #二值变量：指定 BinaryMetropolis  离散变量：指定 Metropolis  连续变量：指定 NUTS
    step = pm.Metropolis()
    trace = pm.sample(100,start=start,step=step,chains=2,cores=8)

print (trace['R'].shape) #直接用于推荐
with BMF_model:
    ppc = pm.sample_posterior_predictive(trace, progressbar=True)
print (ppc['Y'].shape) #直接用于推荐

logp = -7.7694e+08, ||grad|| = 1.0495e+07: 100%|██████████| 9/9 [00:00<00:00, 41.16it/s]   
Only 100 samples in chain.
Multiprocess sampling (2 chains in 8 jobs)
CompoundStep
>Metropolis: [Q]
>Metropolis: [P]
Sampling 2 chains: 100%|██████████| 1200/1200 [00:04<00:00, 299.73draws/s]
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective samples is smaller than 10% for some parameters.
  6%|▌         | 11/200 [00:00<00:01, 102.93it/s]

(200, 11, 698)


100%|██████████| 200/200 [00:02<00:00, 86.05it/s]

(200, 11, 698)


In [11]:
#矩阵分解R=PQ，推荐概率模型MCMC采样-似然函数是Bernoulli
import theano
import pymc3 as pm
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error
import theano.tensor as tt
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder

#1.数据集处理
#http://files.grouplens.org/datasets/movielens/ml-20m-README.html
#the following format of file ratings.csv: userId,movieId,rating,timestamp
#The lines within this file are ordered first by userId, then, within user, by movieId.
#Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).
#Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.
data = pd.read_csv("/data/fjsdata/BayesianRS/ml-20m/ratings.csv",sep=',',low_memory=False,iterator =True)
data = data.get_chunk(100)
#将userId和movieId全部标准编号
le = LabelEncoder()
data = data.apply(le.fit_transform)
data['rating'] = 1
#抽样10%比例测试
test = data.sample(frac=0.1)
#2.构建概率模型
uNum = len(data['userId'].unique())#统计用户数
iNum = len(data['movieId'].unique())#统计电影数
mean= data['rating'].max()/2 #正态分布的均值和方差
k = 100 #隐因子数
X_input = theano.shared(data[['userId','movieId']].values)#转numpy array
Y_output = theano.shared(data['rating'].values)#转numpy array
with pm.Model() as BMF_model:
    # Creating the model
    P = pm.Normal('P', mu=mean, sd=mean, shape=(uNum,k))
    Q = pm.Normal('Q', mu=mean, sd=mean, shape=(k,iNum))
    R = tt.dot(P,Q)
    rY = []
    for row in X_input.get_value(): # 获取每行的值
        rr = R[int(row[0])][int(row[1])]#userId=0,movieId=1
        rY.append(rr)
    rY = pm.Deterministic('rY',pm.math.sigmoid(rY))
    Y = pm.Bernoulli('Y', rY, observed=Y_output.get_value())
#3.后验分布计算  
with BMF_model:        
    start=pm.find_MAP()  # 参数初猜
    #二值变量：指定 BinaryMetropolis  离散变量：指定 Metropolis  连续变量：指定 NUTS
    step = pm.Metropolis()
    trace = pm.sample(100,start=start,step=step,chains=2,cores=8)
    
#4.后验预测  
#X_input.set_value(test[['userId','movieId']].values)#转numpy array
#Y_output.set_value(test['rating'].values)
with BMF_model:
    ppc = pm.sample_posterior_predictive(trace)
    pred = ppc['Y'].mean(axis=0)
    print(pred)
    
print ('RMSE：%f'% mean_squared_error(Y_output.get_value(),pred))

logp = -2,280.5, ||grad|| = 0: 100%|██████████| 2/2 [00:00<00:00, 70.19it/s]
Only 100 samples in chain.
Multiprocess sampling (2 chains in 8 jobs)
CompoundStep
>Metropolis: [Q]
>Metropolis: [P]
Sampling 2 chains: 100%|██████████| 1200/1200 [00:03<00:00, 365.28draws/s]
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective samples is smaller than 10% for some parameters.
100%|██████████| 200/200 [00:00<00:00, 2457.03it/s]

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
RMSE：0.000000
